In [ ]:
!pip install pillow

In [ ]:
import os
import re
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from tqdm import tqdm
from PIL import Image as PILImage  
from io import BytesIO

# Function to open the image for insertion into Excel
def open_image(image_path, max_width=1200, max_height=1000):
    try:
        img = PILImage.open(image_path)  # Open the image using PIL
        img_width, img_height = img.size
        scale = min(max_width / img_width, max_height / img_height) # Calculate the resizing scale to maintain aspect ratio
        new_width = int(img_width * scale) # Calculate new dimensions
        new_height = int(img_height * scale) 
        img = img.resize((new_width, new_height), PILImage.Resampling.LANCZOS) # Resize the image
        img_byte_arr = BytesIO() # Convert the resized image to a byte stream (in memory) for use in openpyxl
        img.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)  # Reset pointer to the beginning of the byte stream
        return Image(img_byte_arr) # Return the image for insertion into Excel
    except FileNotFoundError:
        print(f"File {image_path} not found.")
        return None
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return None

# Main processing function to load and process files
def process_files(directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    files = os.listdir(directory)
    baseline_file = None
    for filename in files:
        if "baseline" in filename.lower():
            baseline_file = filename
            break

    if not baseline_file:
        raise ValueError("Baseline file not found. Ensure a file containing 'baseline' in its name exists.")

    image_directory = 'Output Path' 
    image_paths = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.endswith('.png')]
    
    wb = load_workbook('Output Path/combined_output.xlsx')  # Load the Excel file
    ws_images = wb.create_sheet('Visuals')  
    current_row = 2  # Initialize row position
    current_column = 2
    
    with tqdm(total=len(image_paths), desc="Inserting Images into Excel", unit="image", colour="cyan") as pbar:
        for i, image_path in enumerate(image_paths):
            img = open_image(image_path, max_width=1200, max_height=1000)

            if img:
                cell_position = f"{chr(64 + current_column)}{current_row}"
                ws_images.add_image(img, cell_position)
                # Adjust row and column for the next image
                current_column += 50  # Move to the next column
                if current_column > 1:  # Adjust number of columns per row 
                    current_column = 2  # Reset to column D
                    current_row += 50  # Move to the next row
            else:
                print(f"Image {image_path} could not be opened.")  # Debugging log
            pbar.update(1)
    
    run_match = re.search(r"pacsimulate_(\d+)", baseline_file)  # Extracting run number 
    run_number = run_match.group(1)

    final_filename = f"Carton_Output_run_{run_number}.xlsx" # Define the new output file name
    final_path = os.path.join(output_directory, final_filename)

    wb.save(final_path)  
    print(f"Final file saved with visuals on a separate sheet: {final_filename}")

if __name__ == "__main__":
    directory = "Upload Path"  
    output_directory = "Output Path"  
    process_files(directory, output_directory)